In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

# 1. load model
model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

# For retrieval you need to pass this prompt.
query = "Represent this sentence for searching relevant passages: A man is eating a piece of bread"

docs = [
    query,
    "A man is eating food.",
    "A man is eating pasta.",
    "The girl is carrying a baby.",
    "A man is riding a horse.",
]

# # 2. Encode
# embeddings = model.encode(docs)

# similarities = cos_sim(embeddings[0], embeddings[1:])
# print('similarities:', similarities)

/root/.cache/pypoetry/virtualenvs/aana-XDlPP_xZ-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# model = model.to('cuda')

In [4]:
embeddings = model.encode(docs)

In [6]:
type(embeddings)

numpy.ndarray

In [2]:
from aana.api.sdk import AanaSDK
from aana.api.sdk import get_deployment


aana_sdk = AanaSDK(port=8000)

/root/.cache/pypoetry/virtualenvs/aana-XDlPP_xZ-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-11 11:29:43,525	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


2024-04-11 11:29:45,082	INFO worker.py:1540 -- Connecting to existing Ray cluster at address: 172.17.0.3:49821...
2024-04-11 11:29:45,094	INFO worker.py:1715 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


(ServeReplica:vllm_llama2_7b_chat_deployment:VLLMDeployment pid=55423) INFO 04-11 12:49:00 async_llm_engine.py:433] Received request 386306b7bee449a4a816b101297d82f3: prompt: None, prefix_pos: None,sampling_params: SamplingParams(n=1, best_of=1, presence_penalty=0.0, frequency_penalty=0.0, repetition_penalty=1.0, temperature=0.0, top_p=1.0, top_k=-1, min_p=0.0, seed=None, use_beam_search=False, length_penalty=1.0, early_stopping=False, stop=[], stop_token_ids=[], include_stop_str_in_output=False, ignore_eos=False, max_tokens=1024, logprobs=None, prompt_logprobs=None, skip_special_tokens=True, spaces_between_special_tokens=True), prompt_token_ids: [1, 29871, 13, 1, 518, 25580, 29962, 11221, 278, 1494, 2472, 29892, 1234, 278, 1139, 2114, 1474, 515, 278, 2793, 366, 505, 29889, 13, 13, 3644, 727, 338, 694, 2472, 297, 278, 4333, 29892, 1827, 393, 322, 2832, 873, 26506, 304, 1234, 278, 1139, 29889, 13, 13, 2677, 29901, 13, 13, 268, 1205, 727, 526, 14568, 310, 4828, 411, 7933, 4315, 29889, 13

(ServeReplica:vllm_llama2_7b_chat_deployment:VLLMDeployment pid=55423) INFO 2024-04-11 12:49:12,781 vllm_llama2_7b_chat_deployment_VLLMDeployment rfEPGC replica.py:772 - GENERATE OK 12731.5ms


(ServeReplica:vllm_llama2_7b_chat_deployment:VLLMDeployment pid=55423) INFO 04-11 12:51:52 async_llm_engine.py:433] Received request 4245904ff5f943bdafbafdd40e3c1046: prompt: None, prefix_pos: None,sampling_params: SamplingParams(n=1, best_of=1, presence_penalty=0.0, frequency_penalty=0.0, repetition_penalty=1.0, temperature=0.0, top_p=1.0, top_k=-1, min_p=0.0, seed=None, use_beam_search=False, length_penalty=1.0, early_stopping=False, stop=[], stop_token_ids=[], include_stop_str_in_output=False, ignore_eos=False, max_tokens=1024, logprobs=None, prompt_logprobs=None, skip_special_tokens=True, spaces_between_special_tokens=True), prompt_token_ids: [1, 29871, 13, 1, 518, 25580, 29962, 11221, 278, 1494, 2472, 29892, 1234, 278, 1139, 2114, 1474, 515, 278, 2793, 366, 505, 29889, 13, 13, 3644, 727, 338, 694, 2472, 297, 278, 4333, 29892, 1827, 393, 322, 2832, 873, 26506, 304, 1234, 278, 1139, 29889, 13, 13, 2677, 29901, 13, 13, 268, 1205, 727, 526, 14568, 310, 4828, 411, 7933, 4315, 29889, 13

(ServeReplica:vllm_llama2_7b_chat_deployment:VLLMDeployment pid=55423) INFO 2024-04-11 12:52:04,807 vllm_llama2_7b_chat_deployment_VLLMDeployment rfEPGC replica.py:772 - GENERATE OK 12458.5ms


In [3]:
from aana.deployments.sentence_transformer_deployment import (
    SentenceTransformerConfig,
    SentenceTransformerDeployment,
)

sentence_transformer_deployment = SentenceTransformerDeployment.options(
    num_replicas=1,
    max_concurrent_queries=1000,
    ray_actor_options={"num_gpus": 0.25},
    user_config=SentenceTransformerConfig(
        model="mixedbread-ai/mxbai-embed-large-v1",
        batch_size=4,
        num_processing_threads=2,
    ).model_dump(),
)

aana_sdk.register_deployment(
    "mxbai_embed_large_v1_deployment", sentence_transformer_deployment
)

/root/.cache/pypoetry/virtualenvs/aana-XDlPP_xZ-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_dir" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ('settings_',)`.
  warnings.warn(


(ServeController pid=53030) INFO 2024-04-11 11:29:51,130 controller 53030 deployment_state.py:1547 - Deploying new version of deployment SentenceTransformerDeployment in application 'mxbai_embed_large_v1_deployment'. Setting initial target number of replicas to 1.
(ServeController pid=53030) INFO 2024-04-11 11:29:51,234 controller 53030 deployment_state.py:1831 - Adding 1 replica to deployment SentenceTransformerDeployment in application 'mxbai_embed_large_v1_deployment'.
(ServeReplica:mxbai_embed_large_v1_deployment:SentenceTransformerDeployment pid=104002) /root/.cache/pypoetry/virtualenvs/aana-XDlPP_xZ-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_dir" has conflict with protected namespace "model_".
(ServeReplica:mxbai_embed_large_v1_deployment:SentenceTransformerDeployment pid=104002) 
(ServeReplica:mxbai_embed_large_v1_deployment:SentenceTransformerDeployment pid=104002) You may be able to resolve this warning by setting `model_co

In [13]:
handle = get_deployment("mxbai_embed_large_v1_deployment")

In [5]:
docs = [
    "I am a sentence for which I would like to get its embedding.",
    "And I am another sentence.",
    "I am a sentence for which I would like to get its embedding.",
    "And I am another sentence.",
    "I am a sentence for which I would like to get its embedding.",
    "And I am another sentence.",
]

In [7]:
output = await handle.embed_batch.remote(sentences=docs)

In [10]:
output["embedding"].shape

(6, 1024)